In [ ]:
import csv
import re
import os

log_dir = "./log_data_2"
csv_output_path = "./h3_summary_2.csv"

filename_pattern = r"e(\d+)_(\w+)_hot_(\w+)_cold_(\w+)\.log"

# 기존 stat key (단순 COUNT만 추출)
target_keys = [
    # WAF
    'rocksdb.flush.write.bytes',
    'rocksdb.compact.write.bytes',
    'rocksdb.bytes.written',

    # RAF
    'rocksdb.number.keys.read', # read count
    'rocksdb.bytes.read',       # read bytes
    

    # Cache Hit/Miss
    'rocksdb.block.cache.hit',
    'rocksdb.block.cache.miss',
    "rocksdb.memtable.hit",
    "rocksdb.memtable.miss",

    # Latency
    'rocksdb.db.get.micros',
    'rocksdb.db.write.micros',
    'rocksdb.db.seek.micros',  

    # time
    'rocksdb.compaction.total.time.cpu_micros'  
]

# 헤더 구성
latency_fields = [
    'get.P50', 'get.P95', 'get.P99', 'get.P100', 'get.COUNT', 'get.SUM', 'get.AVG',
    'write.P50', 'write.P95', 'write.P99', 'write.P100', 'write.COUNT', 'write.SUM', 'write.AVG',
]
header = [
    'trial', 'work', 'hot_compaction', 'cold_compaction',
    'time(s)', 'hot_column_key', 'default_column_key'
] + target_keys + latency_fields

rows = []

for log_file_name in os.listdir(log_dir):
    if not log_file_name.endswith('.log'):
        continue

    match = re.match(filename_pattern, log_file_name)
    if not match:
        print(f"Filename {log_file_name} does not match the expected pattern.")
        continue

    trial = match.group(1)
    work = match.group(2)
    hot_compaction = match.group(3)
    cold_compaction = match.group(4)

    stats_dict = {key: 0 for key in target_keys}
    latency_dict = {key: 0 for key in latency_fields}
    time_sec = 0
    hot_col_keys = 0
    default_col_keys = 0

    log_file_path = os.path.join(log_dir, log_file_name)

    with open(log_file_path, "r") as f:
        for line in f:
            line = line.strip()

            time_match = re.match(r"총 소요시간: (\d+(?:\.\d+)?)초", line)
            if time_match:
                time_sec = float(time_match.group(1))
                continue

            hot_match = re.match(r"hot 컬럼에 저장된 키 수: (\d+)", line)
            if hot_match:
                hot_col_keys = int(hot_match.group(1))
                continue

            default_match = re.match(r"default 컬럼에 저장된 키 수: (\d+)", line)
            if default_match:
                default_col_keys = int(default_match.group(1))
                continue

            # 일반 stat 추출
            stat_match = re.match(r"(rocksdb\.[\w\.]+)\s+COUNT\s*:\s*(\d+)", line)
            if stat_match:
                key = stat_match.group(1)
                value = int(stat_match.group(2))
                if key in stats_dict:
                    stats_dict[key] = value

            # Latency: get
            get_match = re.match(r"rocksdb\.db\.get\.micros\s+P50\s*:\s*([\d\.]+)\s+P95\s*:\s*([\d\.]+)\s+P99\s*:\s*([\d\.]+)\s+P100\s*:\s*([\d\.]+)\s+COUNT\s*:\s*(\d+)\s+SUM\s*:\s*(\d+)", line)
            if get_match:
                p50, p95, p99, p100, count, total = map(float, get_match.groups())
                latency_dict.update({
                    'get.P50': p50,
                    'get.P95': p95,
                    'get.P99': p99,
                    'get.P100': p100,
                    'get.COUNT': int(count),
                    'get.SUM': int(total),
                    'get.AVG': total / count if count > 0 else 0
                })

            # Latency: write
            write_match = re.match(r"rocksdb\.db\.write\.micros\s+P50\s*:\s*([\d\.]+)\s+P95\s*:\s*([\d\.]+)\s+P99\s*:\s*([\d\.]+)\s+P100\s*:\s*([\d\.]+)\s+COUNT\s*:\s*(\d+)\s+SUM\s*:\s*(\d+)", line)
            if write_match:
                p50, p95, p99, p100, count, total = map(float, write_match.groups())
                latency_dict.update({
                    'write.P50': p50,
                    'write.P95': p95,
                    'write.P99': p99,
                    'write.P100': p100,
                    'write.COUNT': int(count),
                    'write.SUM': int(total),
                    'write.AVG': total / count if count > 0 else 0
                })

    row = [
        trial, work, hot_compaction, cold_compaction,
        time_sec, hot_col_keys, default_col_keys
    ] + [stats_dict[key] for key in target_keys] + [latency_dict[key] for key in latency_fields]
    rows.append(row)

with open(csv_output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(rows)

print(f"CSV summary saved to {csv_output_path}")

CSV summary saved to ./h3_summary_2.csv


In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.max_rows', None)  # 모든 행 표시

# CSV 파일 읽기
df = pd.read_csv("./h3_summary_2.csv")

df

,trial,work,hot_compaction,cold_compaction,time(s),hot_column_key,default_column_key,rocksdb.flush.write.bytes,rocksdb.compact.write.bytes,rocksdb.bytes.written,rocksdb.number.keys.read,rocksdb.bytes.read,rocksdb.block.cache.hit,rocksdb.block.cache.miss,rocksdb.memtable.hit,rocksdb.memtable.miss,rocksdb.db.get.micros,rocksdb.db.write.micros,rocksdb.db.seek.micros,rocksdb.compaction.total.time.cpu_micros,get.P50,get.P95,get.P99,get.P100,get.COUNT,get.SUM,get.AVG,write.P50,write.P95,write.P99,write.P100,write.COUNT,write.SUM,write.AVG
0,3,write,level,level,230.716,193887,250603,803700692,4848802592,16407311441,1806113,7282524160,5463709,7050007,3777,1802336,0,0,0,73143524,2.564519,11.392981,20.112287,804.0,1806113,8608731,4.766441,8.575411,14.380769,29.787064,4538617.0,1000000,229110925,229.110925
1,3,write,universal,level,268.855,193868,250184,803834193,3751794273,16407310871,1806132,7275347968,7655390,5699702,3780,1802352,0,0,0,56259276,3.065318,15.920499,29.454693,808.0,1806132,10323691,5.715912,8.463245,14.143125,26.525009,4163063.0,1000000,267328410,267.328410
2,2,write,universal,level,290.198,193982,250544,803983344,3901581332,16407310588,1806018,7283113984,7383986,5862333,3775,1802243,0,0,0,57519898,3.264258,19.987719,33.355354,797.0,1806018,11225240,6.215464,8.537124,14.453999,34.847070,4372253.0,1000000,288650252,288.650252
3,2,read,level,universal,18.913,0,0,0,3093595,0,1000000,12653805568,1162824,2025128,0,1000000,0,0,0,0,9.452199,46.592771,62.517001,787.0,1000000,17727709,17.727709,0.000000,0.000000,0.000000,0.0,0,0,0.000000
4,4,write,universal,level,221.116,193978,250500,803903921,3796712352,16407311428,1806022,7282327552,7517332,5732856,4971,1801051,0,0,0,56186669,3.373302,20.127374,35.643018,769.0,1806022,11727071,6.493316,8.639435,14.638072,33.528841,3407552.0,1000000,219547688,219.547688
5,4,write,level,level,196.439,193939,250577,803789859,4855601295,16407310351,1806061,7282950144,4914125,7087445,3777,1802284,0,0,0,72173893,1.938029,13.286025,21.953603,192.0,1806061,8664684,4.797559,8.502900,14.326217,28.019544,3125712.0,1000000,194959431,194.959431
6,1,write,universal,level,193.056,193814,250488,803888051,3799392866,16407311067,1806186,7279443968,6380691,5761089,3770,1802416,0,0,0,57280764,2.654851,15.889428,29.162702,818.0,1806186,9667905,5.352663,8.573284,14.531181,36.239024,2910802.0,1000000,191435368,191.435368
7,4,write,level,universal,207.012,193968,250191,804113783,3606647340,16407310862,1806032,7277101056,8631229,5526361,3777,1802255,0,0,0,55419550,3.386422,14.758983,22.661979,815.0,1806032,10633506,5.887773,8.838353,14.715929,32.709138,3272334.0,1000000,205403609,205.403609
8,1,write,level,universal,234.981,194069,249944,803879736,3809431422,16407311406,1805931,7274708992,6406672,5781933,5030,1800901,0,0,0,57082035,3.005860,14.663067,21.908086,802.0,1805931,9546053,5.285946,8.506973,14.352917,32.187801,4209739.0,1000000,233529423,233.529423
9,2,read,universal,universal,26.274,0,0,0,3091042,0,1000000,12655804416,1163525,2930018,0,1000000,0,0,0,0,23.936973,47.698821,65.454706,834.0,1000000,24990799,24.990799,0.000000,0.000000,0.000000,0.0,0,0,0.000000


In [3]:
import pandas as pd

# case 분류 함수
def classify_case(row):
    hot = row.get('hot_compaction', '').lower()
    cold = row.get('cold_compaction', '').lower()

    if hot == 'level' and cold == 'level':
        return 0
    elif hot == 'universal' and cold == 'universal':
        return 1
    elif hot == 'level' and cold == 'universal':
        return 2
    elif hot == 'universal' and cold == 'level':
        return 3
    else:
        return -1  # 알 수 없는 조합 (예외 처리용)

df_waf = df.copy()

# WAF 계산 (0으로 나눌 경우 방지 및 NaN은 0으로 처리)
df_waf['WAF'] = df.apply(
    lambda row: (row.get('rocksdb.flush.write.bytes', 0) + row.get('rocksdb.compact.write.bytes', 0)) / row['rocksdb.bytes.written']
    if row['rocksdb.bytes.written'] != 0 else 0,
    axis=1
)

# RAF 계산 (0으로 나눌 경우 방지 및 NaN은 0으로 처리)
df_waf['RAF'] = df.apply(
    lambda row: (
        (row.get('rocksdb.number.keys.read', 0) * 16 * 1024) / row['rocksdb.bytes.read']
        if row['rocksdb.bytes.read'] != 0 else 0
    ),
    axis=1
)


# Cache Hit Ratio 계산 (0으로 나눌 경우 방지 및 NaN은 0으로 처리)
df_waf['cache_hit_ratio'] = df.apply(
    lambda row: row.get('rocksdb.block.cache.hit', 0) / (row.get('rocksdb.block.cache.hit', 0) + row.get('rocksdb.block.cache.miss', 0))
    if (row.get('rocksdb.block.cache.hit', 0) + row.get('rocksdb.block.cache.miss', 0)) != 0 else 0,
    axis=1
)

df_waf['memtable_hit_ratio'] = df.apply(
    lambda row: row.get('rocksdb.memtable.hit', 0) / (row.get('rocksdb.memtable.hit', 0) + row.get('rocksdb.memtable.miss', 0))
    if (row.get('rocksdb.memtable.hit', 0) + row.get('rocksdb.memtable.miss', 0)) != 0 else 0,
    axis=1
)

# Throughput 계산 (0으로 나눌 경우 방지 및 NaN은 0으로 처리)
df_waf['throughput'] = df.apply(
    lambda row: row.get('rocksdb.bytes.written', 0) / row['time(s)']
    if row['time(s)'] != 0 else 0,
    axis=1
)

# case 분류
df_waf['case'] = df.apply(classify_case, axis=1)

# NaN 값이 있을 경우 최종적으로 0으로 채움
df_waf.fillna(0, inplace=True)

# 저장
df_waf.to_csv('modified_h3_summary_2.csv', index=False)

In [4]:
df_waf

,trial,work,hot_compaction,cold_compaction,time(s),hot_column_key,default_column_key,rocksdb.flush.write.bytes,rocksdb.compact.write.bytes,rocksdb.bytes.written,rocksdb.number.keys.read,rocksdb.bytes.read,rocksdb.block.cache.hit,rocksdb.block.cache.miss,rocksdb.memtable.hit,rocksdb.memtable.miss,rocksdb.db.get.micros,rocksdb.db.write.micros,rocksdb.db.seek.micros,rocksdb.compaction.total.time.cpu_micros,get.P50,get.P95,get.P99,get.P100,get.COUNT,get.SUM,get.AVG,write.P50,write.P95,write.P99,write.P100,write.COUNT,write.SUM,write.AVG,WAF,RAF,cache_hit_ratio,memtable_hit_ratio,throughput,case
0,3,write,level,level,230.716,193887,250603,803700692,4848802592,16407311441,1806113,7282524160,5463709,7050007,3777,1802336,0,0,0,73143524,2.564519,11.392981,20.112287,804.0,1806113,8608731,4.766441,8.575411,14.380769,29.787064,4538617.0,1000000,229110925,229.110925,0.344511,4.063338,0.436618,0.002091,7.111475e+07,0
1,3,write,universal,level,268.855,193868,250184,803834193,3751794273,16407310871,1806132,7275347968,7655390,5699702,3780,1802352,0,0,0,56259276,3.065318,15.920499,29.454693,808.0,1806132,10323691,5.715912,8.463245,14.143125,26.525009,4163063.0,1000000,267328410,267.328410,0.277658,4.067389,0.573219,0.002093,6.102662e+07,3
2,2,write,universal,level,290.198,193982,250544,803983344,3901581332,16407310588,1806018,7283113984,7383986,5862333,3775,1802243,0,0,0,57519898,3.264258,19.987719,33.355354,797.0,1806018,11225240,6.215464,8.537124,14.453999,34.847070,4372253.0,1000000,288650252,288.650252,0.286797,4.062795,0.557437,0.002090,5.653833e+07,3
3,2,read,level,universal,18.913,0,0,0,3093595,0,1000000,12653805568,1162824,2025128,0,1000000,0,0,0,0,9.452199,46.592771,62.517001,787.0,1000000,17727709,17.727709,0.000000,0.000000,0.000000,0.0,0,0,0.000000,0.000000,1.294788,0.364756,0.000000,0.000000e+00,2
4,4,write,universal,level,221.116,193978,250500,803903921,3796712352,16407311428,1806022,7282327552,7517332,5732856,4971,1801051,0,0,0,56186669,3.373302,20.127374,35.643018,769.0,1806022,11727071,6.493316,8.639435,14.638072,33.528841,3407552.0,1000000,219547688,219.547688,0.280400,4.063243,0.567338,0.002752,7.420228e+07,3
5,4,write,level,level,196.439,193939,250577,803789859,4855601295,16407310351,1806061,7282950144,4914125,7087445,3777,1802284,0,0,0,72173893,1.938029,13.286025,21.953603,192.0,1806061,8664684,4.797559,8.502900,14.326217,28.019544,3125712.0,1000000,194959431,194.959431,0.344931,4.062983,0.409457,0.002091,8.352369e+07,0
6,1,write,universal,level,193.056,193814,250488,803888051,3799392866,16407311067,1806186,7279443968,6380691,5761089,3770,1802416,0,0,0,57280764,2.654851,15.889428,29.162702,818.0,1806186,9667905,5.352663,8.573284,14.531181,36.239024,2910802.0,1000000,191435368,191.435368,0.280563,4.065221,0.525515,0.002087,8.498731e+07,3
7,4,write,level,universal,207.012,193968,250191,804113783,3606647340,16407310862,1806032,7277101056,8631229,5526361,3777,1802255,0,0,0,55419550,3.386422,14.758983,22.661979,815.0,1806032,10633506,5.887773,8.838353,14.715929,32.709138,3272334.0,1000000,205403609,205.403609,0.268829,4.066184,0.609654,0.002091,7.925778e+07,2
8,1,write,level,universal,234.981,194069,249944,803879736,3809431422,16407311406,1805931,7274708992,6406672,5781933,5030,1800901,0,0,0,57082035,3.005860,14.663067,21.908086,802.0,1805931,9546053,5.285946,8.506973,14.352917,32.187801,4209739.0,1000000,233529423,233.529423,0.281174,4.067293,0.525628,0.002785,6.982399e+07,2
9,2,read,universal,universal,26.274,0,0,0,3091042,0,1000000,12655804416,1163525,2930018,0,1000000,0,0,0,0,23.936973,47.698821,65.454706,834.0,1000000,24990799,24.990799,0.000000,0.000000,0.000000,0.0,0,0,0.000000,0.000000,1.294584,0.284234,0.000000,0.000000e+00,1


In [5]:
df_waf['trial'].unique()

array([3, 2, 4, 1])